In [1]:
!pip install transformers trl datasets huggingface_hub bitsandbytes wandb tqdm pillow torchvision peft ipywidgets

## Mount Drive (optional)

In [2]:
# HF_CACHE = "/content/drive/MyDrive/hf_cache"

In [3]:
# from google.colab import drive, runtime
# import os
# drive.mount("/content/drive")

# # One shared cache for everything:
# HF_CACHE = "/content/drive/MyDrive/hf_cache"
# !mkdir -p "$HF_CACHE"

# os.environ["HF_HOME"] = HF_CACHE           # generic root
# os.environ["TRANSFORMERS_CACHE"] = HF_CACHE
# os.environ["HF_DATASETS_CACHE"] = HF_CACHE

## Imports

In [4]:
from __future__ import annotations

import contextlib
import os
from pathlib import Path
from typing import Dict, List, Tuple

from tqdm.auto import tqdm

import torch
from torch import Tensor
from torch.nn.functional import log_softmax, softplus
from torch.utils.data import DataLoader
from PIL import Image
from datasets import load_dataset
from transformers import (
    BitsAndBytesConfig,
    LlavaNextForConditionalGeneration,
    LlavaNextProcessor,
    get_cosine_schedule_with_warmup,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import wandb

## Config

In [5]:
MODEL_ID          = "llava-hf/llava-v1.6-mistral-7b-hf"
DATASET_NAME      = "openbmb/RLAIF-V-Dataset"
WANDB_PROJECT     = "llava-qlora-orpo"
OUTPUT_DIR        = "../logs/"

!mkdir -p "$OUTPUT_DIR"
USE_QLORA         = True

TRAIN_BATCH_SIZE  = 2 if USE_QLORA else 1
VAL_BATCH_SIZE    = 8
TEST_BATCH_SIZE   = 8
GRAD_ACC_STEPS    = 4          # effective batch = TRAIN_BATCH_SIZE × GRAD_ACC_STEPS
EPOCHS            = 1
LEARNING_RATE     = 2e-4
WARMUP_RATIO      = 0.03

VAL_RATIO         = 0.05       # 5 % validation
TEST_RATIO        = 0.05       # 5 % test

LORA_R            = 8  if USE_QLORA else 16
LORA_ALPHA        = 16 if USE_QLORA else 32

LORA_DROPOUT      = 0.05
ORPO_BETA         = 0.1

LOG_EVERY_STEPS   = 4
VAL_EVERY_STEPS   = 100

DEVICE = torch.device("cuda")

# Maximum number of tokens for the answer
MAX_ANSWER_TOKENS = 128 # You can adjust this value

## Load the model

In [6]:
processor = LlavaNextProcessor.from_pretrained(MODEL_ID, use_fast=True)
TOKENIZER = processor.tokenizer
EOS_ID = TOKENIZER.eos_token_id

if USE_QLORA:
    # 4‑bit base + gradient‑ckpt
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    base_model = LlavaNextForConditionalGeneration.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto",
    )

    base_model = prepare_model_for_kbit_training(
        base_model, use_gradient_checkpointing=True
    )
    base_model.config.use_cache = False     # must be OFF with grad‑ckpt
else:
    base_model = LlavaNextForConditionalGeneration.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto",
    )
    base_model.config.use_cache = True      # keep cache on


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 22,151,168 || all params: 7,588,898,816 || trainable%: 0.2919


## Create dataloader

In [8]:
@contextlib.contextmanager
def temporary_padding_side(tokenizer, side: str):
    """Temporarily change padding side (left/right) inside a `with` block."""
    original = tokenizer.padding_side
    tokenizer.padding_side = side
    try:
        yield
    finally:
        tokenizer.padding_side = original


def build_prompt_inputs(images: List[Image.Image], questions: List[str]) -> Dict[str, Tensor]:
    """Tokenise the (question + image placeholder) prompt with left‑padding."""
    conversations = [
        [{"role": "user", "content": [{"type": "text", "text": q}, {"type": "image"}]}]
        for q in questions
    ]
    prompts = [processor.apply_chat_template(c, add_generation_prompt=True) for c in conversations]
    encoded = processor(images=images, text=prompts, padding=True, return_tensors="pt")
    return {k: v.to(DEVICE) for k, v in encoded.items()}


def tokenize_answers(texts: List[str], max_length: int | None = None) -> Tuple[Tensor, Tensor]:
    """Right‑pad assistant answers and append EOS."""

    # Reason for the right pad tokenization -
    # later I will concatenate prompt tokens and potential answer tokens,
    # to get logits in one go, without writing a loop.
    # Having pad tokens in the middle seems very confusing
    # and can be misleading and couse errors in the future.

    with temporary_padding_side(TOKENIZER, "right"):
        encoded = TOKENIZER(
            texts,
            padding=True,
            truncation=True,
            add_special_tokens=False,
            return_tensors="pt",
        )
    ids, mask = encoded["input_ids"], encoded["attention_mask"]
    eos_column = torch.full((ids.size(0), 1), EOS_ID, dtype=ids.dtype)
    ids = torch.cat([ids, eos_column], dim=1)
    mask = torch.cat([mask, torch.ones_like(eos_column)], dim=1)

    if max_length is not None:
        # Trim if longer than max_length
        ids = ids[:, :max_length]
        mask = mask[:, :max_length]

    return ids.to(DEVICE), mask.to(DEVICE)


def collate_fn(batch):
    images = [item["image"] for item in batch]
    questions = [item["question"] for item in batch]
    chosen_texts = [item["chosen"] for item in batch]
    rejected_texts = [item["rejected"] for item in batch]

    prompt_inputs = build_prompt_inputs(images, questions)
    chosen_ids, chosen_mask = tokenize_answers(chosen_texts, max_length=MAX_ANSWER_TOKENS)
    rejected_ids, rejected_mask = tokenize_answers(rejected_texts, max_length=MAX_ANSWER_TOKENS)

    return prompt_inputs, chosen_ids, chosen_mask, rejected_ids, rejected_mask

In [9]:
raw_dataset = load_dataset(DATASET_NAME, split="train[:5%]")
first_split = raw_dataset.train_test_split(test_size=VAL_RATIO + TEST_RATIO, seed=42)
train_dataset = first_split["train"]
val_test_dataset = first_split["test"]
val_fraction_of_tmp = VAL_RATIO / (VAL_RATIO + TEST_RATIO)
second_split = val_test_dataset.train_test_split(test_size=1 - val_fraction_of_tmp, seed=42)
val_dataset = second_split["train"]
test_dataset = second_split["test"]

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Get logits

In [10]:
def get_prompt_cache(prompt_inputs: Dict[str, Tensor], adapters_active: bool):
    """Run the prompt once; return `(last_logit, past_kv)`."""
    ctx = model.disable_adapter() if not adapters_active else contextlib.nullcontext()
    training_mode = model.training
    with ctx:
        model.eval()                       # temporarily enable KV cache
        with torch.no_grad():
            out = model(**prompt_inputs, use_cache=True, return_dict=True)
        if training_mode:                  # restore
            model.train()
    return out.logits[:, -1:, :], out.past_key_values

if USE_QLORA:
    # Overwrite get_prompt_cache
    # I hope I'll find a fix for QLoRA later
    def get_prompt_cache(
            prompt_inputs: Dict[str, Tensor],
            adapters_active: bool
    ) -> Tuple[Tensor, Tuple]:

        return None, None

def sequence_logprob_from_cache(prompt_last_logit: Tensor,
                                past_key_values,
                                continuation_ids: Tensor,
                                continuation_mask: Tensor,
                                adapters_active: bool) -> Tensor:
    """Log‑probability of a continuation (incl. EOS) using an existing cache."""
    ctx = model.disable_adapter() if not adapters_active else contextlib.nullcontext()
    with ctx:
        logits_rest = model(
            input_ids=continuation_ids[:, :-1],
            attention_mask=continuation_mask[:, :-1],
            past_key_values=past_key_values,
            use_cache=False,
            pixel_values=None,
            image_sizes=None,
            return_dict=True,
        ).logits  # (B, N‑1, V)
    full_logits = torch.cat([prompt_last_logit, logits_rest], dim=1)  # (B, N, V)
    log_probs = log_softmax(full_logits, dim=-1)
    token_lp = log_probs.gather(2, continuation_ids.unsqueeze(-1)).squeeze(-1)
    return (token_lp * continuation_mask).sum(dim=-1)  # (B,)


def sequence_logprob(prompt_inputs: Dict[str, Tensor],
                     continuation_ids: Tensor,
                     continuation_mask: Tensor,
                     adapters_active: bool) -> Tensor:
    """
    • Use cache when we have one.
    • Otherwise: run full forward.
      ─ reference policy  -> no grads
      ─ θ‑policy          -> keep grads
    """
    last_logit, pkv = get_prompt_cache(prompt_inputs, adapters_active)

    if last_logit is not None and pkv is not None:      # fast cache path
        return sequence_logprob_from_cache(
            last_logit, pkv, continuation_ids, continuation_mask,
            adapters_active=adapters_active
        )

    # ---------- fallback: no cache ----------
    ctx_adapter = model.disable_adapter() if not adapters_active else contextlib.nullcontext()
    ctx_grad    = torch.no_grad() if not adapters_active else contextlib.nullcontext()

    with ctx_adapter, ctx_grad:
        full_ids  = torch.cat([prompt_inputs["input_ids"],
                               continuation_ids[:, :-1]], dim=1)
        full_mask = torch.cat([prompt_inputs["attention_mask"],
                               continuation_mask[:, :-1]], dim=1)

        logits = model(
            input_ids      = full_ids,
            attention_mask = full_mask,
            pixel_values   = prompt_inputs.get("pixel_values"),
            image_sizes    = prompt_inputs.get("image_sizes"),
            use_cache      = False,
            return_dict    = True,
        ).logits[:, -continuation_ids.size(1):, :]      # (B, N, V)

    logp   = log_softmax(logits, -1)
    tok_lp = logp.gather(2, continuation_ids.unsqueeze(-1)).squeeze(-1)
    return (tok_lp * continuation_mask).sum(dim=-1)

In [ ]:
def orpo_loss(lp_theta_c, lp_theta_r, lp_ref_c, lp_ref_r, beta=ORPO_BETA):
    return softplus(-beta * ((lp_theta_c - lp_theta_r) - (lp_ref_c - lp_ref_r))).mean()

# ───────────────────────────────────────────────────────────
# Optimiser, scheduler, wandb
# ───────────────────────────────────────────────────────────
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.95), weight_decay=0.0)
steps_per_epoch = len(train_loader) // GRAD_ACC_STEPS
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    int(steps_per_epoch * EPOCHS * WARMUP_RATIO),
    steps_per_epoch * EPOCHS,
)

wandb.init(project=WANDB_PROJECT, config={k: v for k, v in globals().items() if k.isupper()})

wandb.watch(model, log="gradients", log_freq=LOG_EVERY_STEPS)

best_val = float("inf")   # lower is better for ORPO
best_step = -1


model.train()
acc_steps = 0
running_loss = 0.0  # To accumulate loss for average
for epoch in range(EPOCHS):
    for global_step, batch in tqdm(enumerate(train_loader, 1), total=len(train_loader), desc=f"Epoch {epoch+1}/{EPOCHS}"):
        prompt_inputs, chosen_ids, chosen_mask, rejected_ids, rejected_mask = batch

        # print(
        #     f"""
        #     input_ids {prompt_inputs["input_ids"].size(1)} chosen_ids {chosen_ids.size(1)}, rejected_ids {rejected_ids.size(1)}, total {prompt_inputs["input_ids"].size(1) + max(chosen_ids.size(1), rejected_ids.size(1))}
        #     """,
        #     end=""
        # )

        lp_theta_chosen   = sequence_logprob(prompt_inputs, chosen_ids, chosen_mask, adapters_active=True)
        lp_theta_rejected = sequence_logprob(prompt_inputs, rejected_ids, rejected_mask, adapters_active=True)

        with torch.no_grad():
            lp_ref_chosen   = sequence_logprob(prompt_inputs, chosen_ids, chosen_mask, adapters_active=False)
            lp_ref_rejected = sequence_logprob(prompt_inputs, rejected_ids, rejected_mask, adapters_active=False)


        loss = orpo_loss(lp_theta_chosen, lp_theta_rejected, lp_ref_chosen, lp_ref_rejected) / GRAD_ACC_STEPS
        loss.backward()
        acc_steps += 1
        running_loss += loss.item() * GRAD_ACC_STEPS # Accumulate original loss before division


        if acc_steps == GRAD_ACC_STEPS:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step(); scheduler.step(); optimizer.zero_grad(); acc_steps = 0

            if global_step % LOG_EVERY_STEPS == 0:
                avg_loss = running_loss / LOG_EVERY_STEPS
                wandb.log(
                    {"train/orpo_loss": avg_loss,
                     "lr": scheduler.get_last_lr()[0],
                     "train/lp_theta_chosen": lp_theta_chosen.mean().item(),
                     "train/lp_theta_rejected": lp_theta_rejected.mean().item(),
                     "train/lp_ref_chosen": lp_ref_chosen.mean().item(),
                     "train/lp_ref_rejected": lp_ref_rejected.mean().item()},
                    step=global_step
                )
                running_loss = 0.0 # Reset running loss


            if global_step % VAL_EVERY_STEPS == 0:
                model.eval()
                val_losses = []
                val_lp_theta_chosen_list = []
                val_lp_theta_rejected_list = []
                val_lp_ref_chosen_list = []
                val_lp_ref_rejected_list = []
                with torch.no_grad():
                    for vb in tqdm(val_loader, desc="Validation"):
                        p_in, c_id, c_m, r_id, r_m = vb

                        lt_last, lt_pkv = get_prompt_cache(p_in, adapters_active=True)
                        lr_last, lr_pkv = get_prompt_cache(p_in, adapters_active=False)

                        lt_c = sequence_logprob(p_in, c_id, c_m, adapters_active=True)
                        lt_r = sequence_logprob(p_in, r_id, r_m, adapters_active=True)
                        lr_c = sequence_logprob(p_in, c_id, c_m, adapters_active=False)
                        lr_r = sequence_logprob(p_in, r_id, r_m, adapters_active=False)

                        val_losses.append(orpo_loss(lt_c, lt_r, lr_c, lr_r).item())
                        val_lp_theta_chosen_list.append(lt_c.mean().item())
                        val_lp_theta_rejected_list.append(lt_r.mean().item())
                        val_lp_ref_chosen_list.append(lr_c.mean().item())
                        val_lp_ref_rejected_list.append(lr_r.mean().item())

                val_loss = sum(val_losses) / len(val_losses)
                if val_loss < best_val:
                    best_val = val_loss
                    best_step = global_step
                    ckpt_dir = f"{OUTPUT_DIR}/step_{best_step}"
                    os.makedirs(ckpt_dir, exist_ok=True)
                    model.save_pretrained(ckpt_dir)
                    wandb.run.summary["best_val_loss"] = best_val
                    wandb.run.summary["best_step"] = best_step
                    print(f"★ New best val_loss {best_val:.4f} at step {best_step} — adapters saved to {ckpt_dir}")
 

                wandb.log({
                    "val/orpo_loss": sum(val_losses) / len(val_losses),
                    "val/lp_theta_chosen": sum(val_lp_theta_chosen_list) / len(val_lp_theta_chosen_list),
                    "val/lp_theta_rejected": sum(val_lp_theta_rejected_list) / len(val_lp_theta_rejected_list),
                    "val/lp_ref_chosen": sum(val_lp_ref_chosen_list) / len(val_lp_ref_chosen_list),
                    "val/lp_ref_rejected": sum(val_lp_ref_rejected_list) / len(val_lp_ref_rejected_list)
                    }, step=global_step)
                model.train()

wandb: Currently logged in as: grach0v (cowboy_bebop) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/1:   0%|          | 0/1871 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/venv/main/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/venv/main/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
# Load the best saved model
if best_step != -1:
    best_ckpt_dir = f"{OUTPUT_DIR}/step_{best_step}"
    print(f"Loading best model from step {best_step}: {best_ckpt_dir}")
    model.load_adapter(best_ckpt_dir, adapter_name="best")
    model.set_adapter("best")
else:
    print("No best model found, using current model state")

model.eval()
test_losses = []
with torch.no_grad():
    for tb in test_loader:
        p_in, c_id, c_m, r_id, r_m = tb
        lt_last, lt_pkv = get_prompt_cache(p_in, adapters_active=True)
        lr_last, lr_pkv = get_prompt_cache(p_in, adapters_active=False)

        lt_c = sequence_logprob_from_cache(lt_last, lt_pkv, c_id, c_m, adapters_active=True)
        lt_r = sequence_logprob_from_cache(lt_last, lt_pkv, r_id, r_m, adapters_active=True)
        lr_c = sequence_logprob_from_cache(lr_last, lr_pkv, c_id, c_m, adapters_active=False)
        lr_r = sequence_logprob_from_cache(lr_last, lr_pkv, r_id, r_m, adapters_active=False)

        test_losses.append(orpo_loss(lt_c, lt_r, lr_c, lr_r).item())

wandb.log({"test/orpo_loss": sum(test_losses) / len(test_losses)})

# ───────── save adapters & finish ─────────
model.save_pretrained(OUTPUT_DIR)
wandb.finish()